# Object detection with YOLO: Training and Validation

Installiamo e/o aggiorniamo i pacchetti ultralytics (per usare YOLO) e wandb (per tracciare gli esperimenti ML).

In [ ]:
!pip install ultralytics -q
!pip install -U ultralytics wandb
!pip install python-dotenv

Configuriamo YOLO in modo da abilitare l'integrazione con Weights & Biases (wandb).

In [ ]:
!yolo settings wandb=True

Importiamo le librerie necessarie:

In [ ]:
from ultralytics import YOLO
import wandb
from dotenv import load_dotenv
import os
import pandas as pd


Autentichiamo il nostro script Python con Weights & Biases (wandb) utilizzando l' API key personale.

In [ ]:
load_dotenv()
wandb.login(key=os.environ["WANDB_API_KEY"])

Spacchettiamo il file .zip contenente il dataset:

In [ ]:
#solo su colab
#!unzip -q /content/drive/MyDrive/.zip

Definiamo alcune variabili di configurazione necessarie per eseguire gli esperimenti con YOLO:

A questo punto possiamo avviare la procedura di training e validazione del modello:

In [ ]:
model_name = 'yolo12m.pt'

model = YOLO(model_name)
model.train(
    cfg = 'train_settings_template.yaml'
)

Utilizziamo il modello allenato per fare inferenza sui dati di test:

In [ ]:
results_test = model.predict(source='/content/dataset_finale/test/images', # immagini o video
                             conf=0.25, # imposta una soglia di confidenza
                             device=0, # usa la GPU
                             save=True) #salva le predizioni

Valutiamo quantitativamente il modello sul test set visualizzando le metriche:

In [ ]:
metrics = model.val(data='path',
          imgsz=416 , conf=0.25, #da modificare imgsz in base al modello
          device=0, split='test',
          project = 'path')

results_dict = metrics.results_dict
results_dict['F1'] = 2 * (results_dict['metrics/precision(B)'] * results_dict['metrics/recall(B)']) / (results_dict['metrics/precision(B)'] + results_dict['metrics/recall(B)'])
pd.DataFrame(results_dict, index=[0]).to_csv("path/test_metrics.csv")